In [ ]:
!pip uninstall -y opencv-python opencv-python-headless opencv-contrib-python \
  array-api-compat xarray xarray-einstats albumentations albucore


In [ ]:
%%capture
!pip install -q --upgrade --force-reinstall numpy==1.23.5 scipy==1.10.1
!pip install -q --no-deps opencv-python-headless==4.7.0.72


In [ ]:
%%capture
!pip install -q imgaug==0.4.0 imageio==2.31.1 matplotlib==3.8.4 \
  Pillow==9.5.0 six==1.16.0 PyYAML==6.0.2 tqdm==4.66.4
# timm relies on torch; install timm but don't let it drag extras
!pip install -q --no-deps timm==0.9.16


In [ ]:
import os; os.kill(os.getpid(), 9)


In [ ]:
import torch; print("torch:", torch.__version__)
import numpy as np; print("numpy:", np.__version__)         # -> 1.23.5
import scipy; print("scipy:", scipy.__version__)            # -> 1.10.1
import cv2; print("opencv:", cv2.__version__)               # -> 4.7.0.72
import imgaug; print("imgaug:", imgaug.__version__)         # -> 0.4.0
import timm; print("timm:", timm.__version__)               # -> 0.9.16


In [ ]:
### Upload our Github to your Google Drive and name it SuperPoint-With-Transformer-Backbone ###

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone

In [ ]:
# Train with a CNN VGG backbone. The Synthetic Dataset is created
# in the 1st time you train.
# A mg_syn___.pth file is saved after each batch
%env PYTHONPATH=.
!python train_MagicPoint_CNN.py ./config/magic_point_syn_train_cnn.yaml


In [ ]:
# Train with a Swin-Tiny Transformer backbone (instead of VGG )
# on the Synthetic Dataset
# A mg_syn_swin___.pth file is saved after each batch

#fresh training:
%env PYTHONPATH=.
!python train_MagicPoint_Transformer.py config/magic_point_syn_train_swin_160x120.yaml --run_name swin_tiny_temp

# #resume training:
# %env PYTHONPATH=.
# !python train.py config/magic_point_syn_train_swin_160x120.yaml \
#     --run_name swin_resume --resume export/last.pth

In [ ]:
# Run and display  MagicPoint Transformer on a .jpg sample.
# mg_syn_swin_15_0.176.pth was trained (in the cell above) on the Synthetic dataset
# with a Swin-Tiny Transformer backbone
%cd /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone
%env PYTHONPATH=.
!python -u scripts/infer_one.py \
  --img sample.jpg \
  --ckpt export/mg_syn_swin_15_0.176.pth \
  --out runs/one_out.jpg --out_prob runs/one_prob.jpg \
  --device cuda:0 --H 120 --W 160 \
  --det_thresh 0.015 --nms 4 --topk 20

from IPython.display import Image, display
display(Image('runs/one_out.jpg', width=1200))   # adjust width as you like





In [ ]:
# Download Dataset for Homographic Adaptation
!mkdir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/EUROC
%cd /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/EUROC

!wget http://robotics.ethz.ch/~asl-datasets/ijrr_euroc_mav_dataset/machine_hall/MH_01_easy/MH_01_easy.zip
!unzip -q MH_01_easy.zip && rm MH_01_easy.zip

import os
import glob
import shutil

src_folder = "/content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/data"
dst_folder1 = "/content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/images_last2000"
dst_folder2 = "/content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/images_first1000"


# Create destination folder if it doesn't exist
os.makedirs(dst_folder1, exist_ok=True)
os.makedirs(dst_folder2, exist_ok=True)


# Get all images, sort by numeric part of filename
image_files = sorted(
    glob.glob(os.path.join(src_folder, "*.jpg")) +
    glob.glob(os.path.join(src_folder, "*.png")) +
    glob.glob(os.path.join(src_folder, "*.jpeg")),
    key=lambda x: int(os.path.splitext(os.path.basename(x))[0])  # assumes name is like "123.jpg"
)

# Copy the last  ~ 2000 frames to a sepearte folder (training set)
# Copy the first  ~ 1000 frames to a sepearte folder (test set)
for img_path in image_files[1500:]:
    shutil.copy(img_path, os.path.join(dst_folder1, os.path.basename(img_path)))
for img_path in image_files[:1500]:
    shutil.copy(img_path, os.path.join(dst_folder2, os.path.basename(img_path)))

print(f"Moved images to {dst_folder1}")
print(f"Moved images to {dst_folder2}")

In [ ]:
# Run Homographic Adaptation (this is like "Semi Supervision" - it creates the Ground Truth feature points])
%env PYTHONPATH=.
!python scripts/homographic_adaptation.py \
  --img_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/images_last2000 \
  --ckpt /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/export/mg_syn_swin_15_0.176.pth \
  --out_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/runs/ha_out \
  --H 120 --W 160 \
  --num_aug 32 --agg mean \
  --det_thresh 0.04 --nms 4 --topk 70 \
  --valid_margin 4 --erode_border 4 --save_viz --save_npys --seed 0
#   --img_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/real/test_one \

In [ ]:
# Fine tune the Swin detector on the Homographic Adaptation results
%env PYTHONPATH=.
!python scripts/finetune_magicpoint_ha.py \
  --img_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/images_last2000 \
  --lab_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/runs/ha_out \
  --ckpt export/mg_syn_swin_15_0.176.pth \
  --out_dir export/finetune_swin \
  --log_dir runs/ft_logs \
  --H 120 --W 160 \
  --epochs1 10 --epochs2 10 --lr1 1e-4 --lr2 1e-5


In [ ]:
# Fine tune the CNN detector on the Homographic Adaptation results
%env PYTHONPATH=.
!python scripts/finetune_magicpoint_ha.py \
  --img_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/images_last2000 \
  --lab_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/runs/ha_out \
  --ckpt export/mg_syn_cnn_13_0.136.pth \
  --out_dir export/finetune_cnn \
  --log_dir runs/ft_logs \
  --H 120 --W 160 \
  --epochs1 10 --epochs2 10 --lr1 1e-4 --lr2 1e-5

In [ ]:
# Check detector's, with Transformer backbone, homographic consistency (on the test samples)
%env PYTHONPATH=.
!python scripts/homography_consistency_dir.py \
  --arch swin \
  --img_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/images_first1000 \
  --ckpt export/finetune_swin/best.pth \
  --csv_out runs/homo_dir/swin.csv \
  --H 120 --W 160 --det_thresh 0.02 --nms 4 --topk 50 \
  --px_thresh 3.0 --valid_margin 4 --trials 1 --seed 123

In [ ]:
# Check detector's, with CNN backbone, homographic consistency (on the test samples)
%env PYTHONPATH=.
!python scripts/homography_consistency_dir.py \
  --arch cnn \
  --img_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/images_first1000 \
  --ckpt export/finetune_cnn/best.pth \
  --csv_out runs/homo_dir/cnn.csv \
  --H 120 --W 160 --det_thresh 0.02 --nms 4 --topk 50 \
  --px_thresh 3.0 --valid_margin 4 --trials 1 --seed 123

In [ ]:
# Create feature point tracking on a video, from frame to frame,
# using the Swin detector fine tuned on the Homographic Adaptation results
%env PYTHONPATH=.
!python scripts/track_seq_lk_magicpoint_sxs_firstonly.py \
  --img_dir /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/images_first1000 \
  --out_video /content/drive/MyDrive/SuperPoint-With-Transformer-Backbone/data/Euroc/mav0/cam0/tracks_first50_once.mp4 \
  --ckpt export/finetune_swin/best.pth \
  --det_thresh 0.02 --nms 4 --topk 50 \
  --num_frames 50 --fps 5 --fb_check --fb_thresh 2